In [1]:
import pandas as pd
import numpy as np
import datetime
import time

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

import plotly.graph_objects as go
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html

import ibmdata

In [37]:
DAYSBACK = 365
QUERY = f"""
WITH metro_cte AS(
    SELECT DISTINCT wafer_Id, AVG(dcItem_Value) AS PC_ACI_CD_Logic_60CPP
        --MIN(date) as date, meas_Pd_Id AS step
    FROM DMIW.MeasuredRawFact mrf
    INNER JOIN DMIW_SYSTEMS.MeasuredParm mp ON mp.measuredparmkey = mrf.measuredparmkey
    INNER JOIN DMIW_SYSTEMS.MeasuredWafer mw ON mw.measuredWaferKey = mrf.measuredWaferKey
    WHERE Measurement_date >= (current date - {DAYSBACK} days) 
        AND Tech_id = '7HPP' AND Meas_pd_id = 'KU072615'
        AND meas_Pd_Id <> '' AND dcItem_Value IS NOT null
        AND UCASE(dcItem_Name) NOT IN ('XCOORD', 'YCOORD', 'XSITE', 'YSITE', 'FIELD X', 'FIELD Y', 'SITE X', 'SITE Y', 'SITEID', 'ROW', 'COLUMN') 
        AND (UCASE(rtrim(meas_Pd_Id) || ' ' || dcItem_Name) LIKE '%PC%ACI%CD%(LOGIC%60%' )  
    GROUP BY wafer_Id
    ORDER BY wafer_Id
),
sail_cte AS(
    SELECT DISTINCT wafer_id,
        AVG(CASE WHEN parm_Label = 'SAILALL_H2_0p50to1p05_PerfectYield' THEN yield ELSE NULL END) AS SAILALL_0p50to1p05,
        AVG(CASE WHEN parm_Label = 'SAILALL_H2_0p65to1p05_PerfectYield' THEN yield ELSE NULL END) AS SAILALL_0p65to1p05
    FROM DMIW.PTileWaferFact ptwf
    INNER JOIN DMIW_SYSTEMS.TestParm tp ON ptwf.testparmkey = tp.testparmkey
    INNER JOIN DMIW_SYSTEMS.TestedWafer tw ON ptwf.testedWaferKey = tw.testedWaferKey
    WHERE tw.Last_test_date >= (current date - {DAYSBACK} days)
      AND tw.Tech_id = '7HPP'
      AND tw.Level = 'H2'
      AND tw.Calcdefs in ('SSL11', 'SSL21', 'SSL31')
      AND weighted_Mean IS NOT null AND ABS(weighted_Mean) < 1e25
      AND UCASE(tp.parm_Label) LIKE '%_PERFECTYIELD'
    GROUP BY wafer_id
),
lot_name_cte AS(
    SELECT DISTINCT wafer_id, E.CELL_NAME AS Lot_Name
    FROM DMIW_SYSTEMS.WAFER A,
        DMIW_SYSTEMS.PRODUCT B,
        DMIW_SYSTEMS.EWR_EVENT_FACT C,
        DMIW_SYSTEMS.EWR_EVENT D,
        DMIW_SYSTEMS.EWR_CELL E
    WHERE A.WAFERKEY=C.WAFERKEY
        AND B.PRODUCTKEY=C.PRODUCTKEY
        AND  C.EWR_CELLKEY=E.EWR_CELLKEY
        AND OWNER_USER_ID='hai.zhu@ibm.com'
        AND B.FAMILY_CODE = 'Q5'
        AND EWR_NAME IN ('7HPP_Name_P10')
        AND CELL_ID LIKE '7HPP_Name_P10%'
        AND E.CELL_NAME NOT IN ('Lot0-1','Lot0-2','uncertain')
)
SELECT ln.lot_name, m.wafer_id, m.PC_ACI_CD_Logic_60CPP, s.sailall_0p50to1p05, s.sailall_0p65to1p05
FROM metro_cte m
INNER JOIN sail_cte s ON m.wafer_id = s.wafer_id
INNER JOIN lot_name_cte ln ON ln.wafer_id = m.wafer_id
ORDER BY m.wafer_id
"""
w_df = ibmdata.isdw.query(QUERY)
w_df['sailall_0p50to1p05'] = w_df['sailall_0p50to1p05'].astype(float).round(2)
w_df['sailall_0p65to1p05'] = w_df['sailall_0p65to1p05'].astype(float).round(2)

In [39]:
w_df.to_csv('~/Downloads/wafer.csv')

In [71]:
DAYSBACK = 365
QUERY = f"""
WITH metro_cte AS(
SELECT wafer_Id, unitcell_X - 1 AS retx, unitcell_Y AS rety, dcItem_Value AS PC_ACI_CD_Logic_60CPP
FROM DMIW.MeasuredRawFact mrf
INNER JOIN DMIW_SYSTEMS.MeasuredParm mp ON mp.measuredparmkey = mrf.measuredparmkey
INNER JOIN DMIW_SYSTEMS.Geography g ON g.geographyKey = mrf.geographyKey
INNER JOIN DMIW_SYSTEMS.MeasuredWafer mw ON mw.measuredWaferKey = mrf.measuredWaferKey
WHERE Measurement_date >= (current date - {DAYSBACK} day)
    AND Tech_id = '7HPP' AND Meas_pd_id = 'KU072615'
    AND meas_Pd_Id <> '' AND dcItem_Value IS NOT null
    AND UCASE(dcItem_Name) NOT IN ('XCOORD', 'YCOORD', 'XSITE', 'YSITE', 'FIELD X', 'FIELD Y', 'SITE X', 'SITE Y', 'SITEID', 'ROW', 'COLUMN') 
    AND (UCASE(rtrim(meas_Pd_Id) || ' ' || dcItem_Name) LIKE '%PC%ACI%CD%(LOGIC%60%' )  
ORDER BY lot_Id, wafer_Id
),
sail_cte AS(
SELECT wafer_Id, unitcell_X -1 AS kerf_X, unitcell_Y AS kerf_Y,
    AVG(CASE WHEN parm_Label = 'SAILALL_H2_0p50to1p05_PerfectYield' THEN parmYield ELSE NULL END) AS SAILALL_0p50to1p05,
    AVG(CASE WHEN parm_Label = 'SAILALL_H2_0p65to1p05_PerfectYield' THEN parmYield ELSE NULL END) AS SAILALL_0p65to1p05
FROM DMIW.ChipParmFactR cpfr 
INNER JOIN DMIW_SYSTEMS.TestParm tp ON tp.testParmKey = cpfr.testParmKey 
INNER JOIN DMIW_SYSTEMS.Geography g ON g.geographyKey = cpfr.geographyKey
INNER JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = cpfr.testedWaferKey
WHERE Last_test_date >= (current date - {DAYSBACK} days) AND Tech_id = '7HPP' AND Level = 'H2' AND tw.Calcdefs = 'SSL11' 
    AND cpfr.parmValue IS NOT null AND abs(parmValue) < 1e25 
    AND ( UCASE(tp.parm_Label) like '%SAILALL%' )
GROUP BY wafer_id, unitcell_X, unitcell_Y
)
SELECT m.wafer_id, m.retx, m.rety, PC_ACI_CD_Logic_60CPP, sailall_0p50to1p05, sailall_0p65to1p05
FROM metro_cte m,
sail_cte s
WHERE m.wafer_id = s.wafer_id AND m.retx = s.kerf_x AND m.rety = s.kerf_y
"""
c_df = ibmdata.isdw.query(QUERY)
c_df['sailall_0p50to1p05'] = c_df['sailall_0p50to1p05'].astype(float).round(2)
c_df['sailall_0p65to1p05'] = c_df['sailall_0p65to1p05'].astype(float).round(2)

In [72]:
c_df.to_csv('~/Downloads/chip.csv')